<a href="https://colab.research.google.com/github/weagan/Tiny-Recursive-Models/blob/main/More_tools_MCP_server_accepts_JSON_RPC_requests_and_responds_over_SSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare for Google Cloud Run Deployment

### Subtask:
Create a `main.py` file with the server application and a `Procfile` if necessary, ensuring the server listens on the `PORT` environment variable as required by Google Cloud Run. Provide instructions for deploying the application without using Docker.


# Task
Update the `main.py` file to include two new JSON-RPC 2.0 methods: `list_tools` (which returns a list of available RPC methods) and `add_numbers` (which sums two numerical parameters). Both new methods should also publish their results via SSE. After updating, redeploy the `rpc-sse-service` to Google Cloud Run and then modify the Python test client to call these new methods, verifying their end-to-end functionality, and finally delete the Cloud Run service.

In [1]:
%%writefile main.py

import asyncio
import json
import os
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import StreamingResponse
from pydantic import BaseModel, Field
from typing import Optional, Union, Dict, List, Any, Literal

# Initialize FastAPI application
app = FastAPI()

# --- SSE Message Queue Setup ---
sse_message_queue = asyncio.Queue()

# --- JSON-RPC 2.0 Request Model ---
class JsonRpcRequest(BaseModel):
    jsonrpc: Literal["2.0"] = Field(default="2.0")
    method: str
    params: Optional[Union[List[Any], Dict[str, Any]]] = None
    id: Optional[Union[str, int]] = None

# --- "MCP command" handler functions ---
async def handle_ping(params: Optional[Any]) -> str:
    result = "pong"
    message = {"type": "rpc_result", "method": "ping", "result": result}
    await sse_message_queue.put(json.dumps(message))
    return result

async def handle_greet(params: Optional[Dict[str, Any]]) -> str:
    name = params.get("name", "Guest") if params else "Guest"
    result = f"Hello, {name}!"
    message = {"type": "rpc_result", "method": "greet", "params": params, "result": result}
    await sse_message_queue.put(json.dumps(message))
    return result

async def handle_claude_interaction(params: Optional[Dict[str, Any]]) -> str:
    query = params.get("query", "") if params else ""
    if query:
        result = f"AI placeholder: Processing query '{query}'... (Claude would respond here)Paris?"
    else:
        result = "AI placeholder: No query provided for Claude interaction."
    message = {"type": "rpc_result", "method": "claude.ask", "params": params, "result": result}
    await sse_message_queue.put(json.dumps(message))
    return result

async def handle_list_tools(params: Optional[Any]) -> List[str]:
    """Returns a list of all available RPC methods."""
    available_methods = list(method_handlers.keys())
    message = {"type": "rpc_result", "method": "list_tools", "result": available_methods}
    await sse_message_queue.put(json.dumps(message))
    return available_methods

async def handle_add_numbers(params: Optional[Dict[str, Any]]) -> Union[int, float]:
    """Adds two numerical parameters and returns the result."""
    if not params or "a" not in params or "b" not in params:
        raise HTTPException(status_code=400, detail="Parameters 'a' and 'b' are required for add_numbers.")

    try:
        a = float(params["a"])
        b = float(params["b"])
    except (ValueError, TypeError):
        raise HTTPException(status_code=400, detail="Parameters 'a' and 'b' must be numbers.")

    result = a + b
    message = {"type": "rpc_result", "method": "add_numbers", "params": params, "result": result}
    await sse_message_queue.put(json.dumps(message))
    return result

# Dictionary to map method names to handler functions
method_handlers = {
    "ping": handle_ping,
    "greet": handle_greet,
    "claude.ask": handle_claude_interaction,
    "list_tools": handle_list_tools,
    "add_numbers": handle_add_numbers,
}

# --- Root Endpoint ---
@app.get("/")
async def read_root():
    return {"message": "Hello from FastAPI!"}

# --- RPC Endpoint ---
@app.post("/rpc")
async def json_rpc_endpoint(rpc_request: JsonRpcRequest):
    response_id = rpc_request.id

    if rpc_request.jsonrpc != "2.0":
        return {
            "jsonrpc": "2.0",
            "error": {"code": -32600, "message": "Invalid Request: jsonrpc field must be '2.0'"},
            "id": response_id
        }

    handler = method_handlers.get(rpc_request.method)

    if handler is None:
        return {
            "jsonrpc": "2.0",
            "error": {"code": -32601, "message": "Method not found"},
            "id": response_id
        }

    try:
        result = await handler(rpc_request.params)
        return {
            "jsonrpc": "2.0",
            "result": result,
            "id": response_id
        }
    except HTTPException as e:
        return {
            "jsonrpc": "2.0",
            "error": {"code": -32000, "message": e.detail},
            "id": response_id
        }
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "error": {"code": -32000, "message": str(e)},
            "id": response_id
        }

# --- SSE Event Generator ---
async def event_generator():
    while True:
        message = await sse_message_queue.get() # Get message from the queue
        yield f"data: {message}\n\n"

# --- SSE Endpoint ---
@app.get("/events")
async def sse_endpoint():
    return StreamingResponse(
        event_generator(),
        media_type="text/event-stream",
        headers={
            "Cache-Control": "no-cache",
            "Connection": "keep-alive"
        }
    )

print("Consolidated FastAPI app code with new RPC methods written to main.py")

Writing main.py


**Reasoning**:
Now that the `main.py` file has been created with the consolidated server code, I need to create a `requirements.txt` file listing all the Python dependencies. This file is essential for Google Cloud Run to install the necessary packages when deploying the application.



In [2]:
%%writefile requirements.txt
fastapi
uvicorn
pydantic
httpx

Writing requirements.txt


In [3]:
pip install -r requirements.txt -q

In [4]:
!python main.py #test code

Consolidated FastAPI app code with new RPC methods written to main.py


In [5]:
!gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=DtXvWYnaLL2BtfDCIuvepJpsxDOT3w&prompt=consent&token_usage=remote&access_type=offline&code_challenge=8ujp0OTVPdBvy0UZ572XoQjMoLV-hFBpV980BU8p_yA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab32j91LIM0tiYCAFqyakRDrjtToTKXtXKy3MAQOY46GYnn74ReWil25f1sA0qPT_oRvsQ

You are now logged in as [weagan@gmail.com].
Your current project is [N

In [6]:
!gcloud run deploy rpc-sse-service \
--source . \
--region us-central1 \
--platform managed \
--allow-unauthenticated \
--memory 512Mi \
--project dep-colab-notebooks-cloud-run

Building using Buildpacks and deploying container to Cloud Run service [rpc-sse-service] in project [dep-colab-notebooks-cloud-run] region [us-central1]
Service [rpc-sse-service] revision [rpc-sse-service-00002-5g4] has been deployed and is serving 100 percent of traffic.
Service URL: https://rpc-sse-service-691049916841.us-central1.run.app


## Develop Python Test Client

### Subtask:
Write a Python script that acts as a test client. This client will concurrently perform two main tasks: (1) send JSON-RPC 2.0 commands to the server and (2) listen for and print incoming SSE events from the server to verify end-to-end functionality.


To stop the service from serving traffic without deleting it, you can clear all traffic assignments:

# Task
Update the Python test client to include calls to the `list_tools` and `add_numbers` RPC methods, and then execute the updated client to verify the end-to-end functionality of these new features.

In [7]:
#client code
import asyncio
import json
import httpx

# Server URL (assuming local execution, adjust if server is deployed elsewhere)
SERVER_URL = "https://rpc-sse-service-691049916841.us-central1.run.app" # <--- REPLACE THIS WITH YOUR CLOUD RUN SERVICE URL

async def send_rpc_requests(server_url: str):
    """Sends multiple JSON-RPC 2.0 requests to the server."""
    async with httpx.AsyncClient() as client:
        requests = [
            {
                "jsonrpc": "2.0",
                "method": "ping",
                "id": 1
            },
            {
                "jsonrpc": "2.0",
                "method": "greet",
                "params": {"name": "Alice"},
                "id": 2
            },
            {
                "jsonrpc": "2.0",
                "method": "claude.ask",
                "params": {"query": "What is the capital of France?"},
                "id": 3
            },
            {
                "jsonrpc": "2.0",
                "method": "greet",
                "params": {"name": "Bob"},
                "id": 4
            },
            {
                "jsonrpc": "2.0",
                "method": "list_tools",
                "id": 5
            },
            {
                "jsonrpc": "2.0",
                "method": "add_numbers",
                "params": {"a": 10, "b": 20},
                "id": 6
            },
            {
                "jsonrpc": "2.0",
                "method": "add_numbers",
                "params": {"a": 5.5, "b": 2.3},
                "id": 7
            },
            {
                "jsonrpc": "2.0",
                "method": "add_numbers",
                "params": {"a": "invalid", "b": 20},
                "id": 8
            } # Example of an invalid request
        ]

        print("\n--- Sending RPC Requests ---")
        for req in requests:
            try:
                print(f"Sending RPC request: {req.get('method')} (id: {req.get('id')})")
                response = await client.post(f"{server_url}/rpc", json=req)
                response.raise_for_status()
                # Pretty-print RPC response
                print(f"Received RPC response (id: {req.get('id')}): {json.dumps(response.json(), indent=2)}")
                await asyncio.sleep(1.0) # Small delay between requests
            except httpx.HTTPStatusError as e:
                print(f"RPC request failed for {req.get('method')} (id: {req.get('id')}): {e.response.status_code} - {e.response.text}")
            except httpx.RequestError as e:
                print(f"RPC request failed for {req.get('method')} (id: {req.get('id')}): {e}")
        print("--- Finished Sending RPC Requests ---")

async def listen_for_sse_events(server_url: str):
    """Listen for Server-Sent Events from the server."""
    print("\n--- Listening for SSE Events ---")
    event_count = 0
    max_events = 20 # Listen for a maximum of 20 events to capture new RPC results (Increased from 15)
    timeout_seconds = 60 # Stop listening after 60 seconds (Increased from 30)

    try:
        async with httpx.AsyncClient(timeout=timeout_seconds) as client:
            async with client.stream("GET", f"{server_url}/events", headers={
                "Accept": "text/event-stream",
                "Cache-Control": "no-cache",
                "Connection": "keep-alive"
            }) as response:
                response.raise_for_status()
                print(f"Connected to SSE stream at {server_url}/events")
                async for chunk in response.aiter_bytes():
                    data = chunk.decode().strip()
                    # Split by \n\n to handle multiple events in one chunk or partial events
                    for line in data.split('\n\n'):
                        if line.startswith("data:"):
                            event_data = line[len("data:"):].strip()
                            try:
                                parsed_event = json.loads(event_data)
                                print(f"Received SSE event ({event_count + 1}): {json.dumps(parsed_event, indent=2)}")
                            except json.JSONDecodeError:
                                print(f"Received SSE event ({event_count + 1}) (non-JSON): {event_data}")
                            event_count += 1
                            if event_count >= max_events:
                                print(f"Reached max_events ({max_events}), stopping SSE listener.")
                                return # Use return to exit the function and stop the generator
    except httpx.ConnectError as e:
        print(f"Could not connect to SSE server: {e}. Is the server running at {server_url}?")
    except httpx.ReadTimeout as e:
        print(f"SSE listener timed out after {timeout_seconds} seconds: {e}")
    except httpx.HTTPStatusError as e:
        # Ensure we read the response content if there was an HTTP error
        if e.response.is_error and not e.response.is_closed:
            try:
                error_text = await e.response.aread()
                print(f"SSE connection failed: {e.response.status_code} - {error_text.decode()}")
            except Exception as read_e:
                print(f"SSE connection failed: {e.response.status_code} - Could not read error response: {read_e}")
        else:
            print(f"SSE connection failed: {e.response.status_code} - {e.response.reason_phrase}")
    except Exception as e:
        print(f"An unexpected error occurred while listening to SSE: {e}")
    finally:
        print("--- Stopped Listening for SSE Events ---")

async def main():
    """Runs the RPC requests and SSE listener concurrently."""
    print(f"Starting test client. Server expected at: {SERVER_URL}")
    # Run both tasks concurrently. The SSE listener will run until a timeout or max_events.
    # The RPC sender will complete its requests and then finish.
    await asyncio.gather(
        send_rpc_requests(SERVER_URL),
        listen_for_sse_events(SERVER_URL)
    )
    print("Test client finished.")

# The if __name__ == "__main__" block is commented out for Colab environments
# because asyncio.run() cannot be called from a running event loop.
# Instead, we will directly await main() if in an interactive environment.
# if __name__ == "__main__":
#     # To run this client, ensure your FastAPI server is running on localhost:8000
#     # You can typically start it using: uvicorn main:app --reload
#     asyncio.run(main())

# For interactive environments like Colab, directly await the main function.
await main()

Starting test client. Server expected at: https://rpc-sse-service-691049916841.us-central1.run.app

--- Sending RPC Requests ---
Sending RPC request: ping (id: 1)

--- Listening for SSE Events ---
Connected to SSE stream at https://rpc-sse-service-691049916841.us-central1.run.app/events
Received SSE event (1): {
  "type": "rpc_result",
  "method": "ping",
  "result": "pong"
}
Received RPC response (id: 1): {
  "jsonrpc": "2.0",
  "result": "pong",
  "id": 1
}
Sending RPC request: greet (id: 2)
Received SSE event (2): {
  "type": "rpc_result",
  "method": "greet",
  "params": {
    "name": "Alice"
  },
  "result": "Hello, Alice!"
}
Received RPC response (id: 2): {
  "jsonrpc": "2.0",
  "result": "Hello, Alice!",
  "id": 2
}
Sending RPC request: claude.ask (id: 3)
Received RPC response (id: 3): {
  "jsonrpc": "2.0",
  "result": "AI placeholder: Processing query 'What is the capital of France?'... (Claude would respond here)Paris?",
  "id": 3
}
Received SSE event (3): {
  "type": "rpc_res

In [8]:
#Delete service if desired
# !gcloud run services delete rpc-sse-service \
#   --region us-central1 \
#   --project dep-colab-notebooks-cloud-run

##What's an example of a tool that would stream back over SSE?
A great example of a tool that would specifically benefit from streaming back over SSE is a long-running computation or a real-time monitoring service.

Imagine a tool called process_large_dataset or monitor_system_metrics.

Example: process_large_dataset

When a client invokes an RPC method like process_large_dataset, the server might immediately return an RPC response with a job_id or status: "processing".

However, the actual detailed progress, intermediate results, or completion status of this long-running job wouldn't fit into a single, immediate RPC response. Instead, the server could use SSE to stream updates like:

data: {"job_id": "123", "status": "step1_started", "timestamp": "..."}
data: {"job_id": "123", "progress": "25%", "message": "Extracting features...", "timestamp": "..."}
data: {"job_id": "123", "progress": "75%", "intermediate_result": "partial_data.csv", "timestamp": "..."}
data: {"job_id": "123", "status": "completed", "final_result_url": "/results/job_123", "timestamp": "..."}
Why SSE is perfect for this:

Asynchronous Updates: The client doesn't need to constantly poll the server for updates. The server pushes information as it becomes available.
Real-time Feedback: Users get immediate, continuous feedback on the progress of their request.
Multiple Clients: If multiple clients are interested in the status of job_id: "123", they can all subscribe to the SSE stream and receive the same updates simultaneously.
Long-lived Connection: SSE maintains an open connection, which is more efficient than repeated HTTP requests for continuous updates.


In [9]:
# !gcloud run services update rpc-sse-service \
#   --ingress=internal \
#   --region us-central1 \
#   --project dep-colab-notebooks-cloud-run

In [10]:
# !gcloud run services describe rpc-sse-service \
# --region us-central1 \
# --project dep-colab-notebooks-cloud-run \
# --format='value(spec.template.spec.containers[0].image)'

In [11]:
# !gcloud run deploy rpc-sse-service \
# --image us-central1-docker.pkg.dev/dep-colab-notebooks-cloud-run/cloud-run-source-deploy/rpc-sse-service@sha256:c49394d6c5b63190d620c3b5bbd93bc91167962ce0b7a9c9c5a6b2aa97ea7b7b \
# --region us-central1 \
# --no-traffic \
# --project dep-colab-notebooks-cloud-run

In [12]:
# !gcloud run revisions list --service rpc-sse-service \
#   --region us-central1 \
#   --project dep-colab-notebooks-cloud-run \
#   --limit=1 \
#   --sort-by=CREATE_TIME \
#   --format='value(metadata.name)'

In [13]:
# !gcloud run services update-traffic rpc-sse-service \
#   --to-revisions rpc-sse-service-00003-zxx=100 \
#   --region us-central1 \
#   --project dep-colab-notebooks-cloud-run

In [14]:
# First, get the current image of your service (you might know this already)
# gcloud run services describe <SERVICE_NAME> --region <REGION> --project <PROJECT_ID> --format='value(spec.template.spec.containers[0].image)'

# Then deploy a new revision and explicitly state no traffic should be routed to it
# gcloud run deploy <SERVICE_NAME> \
#   --image <YOUR_SERVICE_IMAGE> \
#   --region <REGION> \
#   --project <PROJECT_ID> \
#   --no-traffic

# !gcloud run deploy rpc-sse-service \
# --source . \
# --region us-central1 \
# --no-traffic \
# --project dep-colab-notebooks-cloud-run

In [15]:
# !gcloud run services delete rpc-sse-service \
#   --region us-central1 \
#   --project dep-colab-notebooks-cloud-run
